<h1>Práctica de Laboratorio: Análisis de Emociones y Sentimiento</h1>
<p><strong>Máster en Bioinformática y Biología Computacional</strong><br>
<strong>Minería de Texto - Curso 2025-26</strong></p>

<p><strong>Integrantes:</strong> [Gonzalo Santana, Tania] y [Parra Gutiérrez, Daniel]</p>


En este ejercicio, trabajaremos con el Procesamiento del Lenguaje Natural (PLN) para analizar las emociones 
expresadas en textos literarios disponibles en Project Gutenberg. El objetivo es construir un sistema que pueda 
identificar y contar las emociones y sentimientos presentes en estas obras. Este ejercicio se enfoca en el uso de 
técnicas avanzadas de PLN, la extracción de información de texto y el procesamiento de lenguaje natural en 
general. 

Para llevar a cabo este ejercicio, se te proporcionará acceso a una serie de recursos y herramientas, incluyendo 
el léxico de emociones NRC (National Research Council), la base de datos léxica WordNet, y la biblioteca de 
Python Beautiful Soup. 

#### Configuración inicial
En esta sección, importamos las bibliotecas necesarias y descargamos los recursos de NLTK.

In [ ]:
#!pip install nltk

In [43]:
# Importaciones necesarias
import nltk
from nltk.corpus import wordnet as wn
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import requests
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from collections import defaultdict, Counter
import re

# Descargar recursos de NLTK
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')  # recursos adicionales si hacen falta

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Tania\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tania\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Tania\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Tania\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [100]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tania\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [106]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

text = "Este es un ejemplo de tokenización."
tokens = word_tokenize(text)
print(tokens)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tania\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\Tania/nltk_data'
    - 'C:\\Users\\Tania\\anaconda3\\nltk_data'
    - 'C:\\Users\\Tania\\anaconda3\\share\\nltk_data'
    - 'C:\\Users\\Tania\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\Tania\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


**Tarea 1**

(1.5 puntos) Cargar en una estructura de datos Python el Word-Emotion Association Lexicon del 
NRC5. Asegúrate de entender cómo se estructura el léxico y cómo se mapean las palabras a las 
emociones. Hay que tener en cuenta que existen varios ficheros con la misma información: un fichero 
con toda la información, un fichero por emoción, etc. Se puede elegir la opción que se estime oportuna. 
Se deberá considerar cómo organizar el léxico en memoria para un acceso rápido durante el análisis. 

##### Carga del léxico de emociones NRC

Vamos a cargar el léxico NRC. Supongamos que hemos descargado el archivo "NRC-Emotion-Lexicon-Wordlevel-v0.92.txt" desde el enlace proporcionado. Este archivo tiene el formato: palabra, emoción, asociación (1 o 0).

Nota: Podemos descargar el archivo manualmente o por código.

###### Descarga por código

In [47]:
import requests

# URL directa del léxico NRC (puede cambiar, pero esta es la habitual)
# url = "https:/mmm/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt"

# Nombre del archivo local
filename = "NRC-Emotion-Lexicon-Wordlevel-v0.92.txt"

# Descargar y guardar
response = requests.get(url)
with open(filename, "w", encoding="utf-8") as f:
    f.write(response.text)

print(f"Archivo descargado y guardado como '{filename}'")

Archivo descargado y guardado como 'NRC-Emotion-Lexicon-Wordlevel-v0.92.txt'


In [72]:
# Cargar el léxico NRC
emolex_file = 'NRC-Emotion-Lexicon-Wordlevel-v0.92.txt'
emolex = {}

with open(emolex_file, 'r') as f:
    for line in f:
        word, emotion, association = line.strip().split('\t')
        if association == '1':
            if word not in emolex:
                emolex[word] = []
            emolex[word].append(emotion)

# Mostrar algunas palabras y sus emociones
list(emolex.items())[:10]

[('abacus', ['trust']),
 ('abandon', ['fear', 'negative', 'sadness']),
 ('abandoned', ['anger', 'fear', 'negative', 'sadness']),
 ('abandonment', ['anger', 'fear', 'negative', 'sadness', 'surprise']),
 ('abba', ['positive']),
 ('abbot', ['trust']),
 ('abduction', ['fear', 'negative', 'sadness', 'surprise']),
 ('aberrant', ['negative']),
 ('aberration', ['disgust', 'negative']),
 ('abhor', ['anger', 'disgust', 'fear', 'negative'])]

In [74]:
def load_emolex(file_path):
    """
    Carga el léxico de emociones NRC desde un archivo
    Retorna: dict con estructura {palabra: [emociones]}
    """
    emolex = {}
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                parts = line.strip().split('\t')
                if len(parts) == 3:
                    word, emotion, value = parts
                    if value == '1':
                        if word not in emolex:
                            emolex[word] = []
                        emolex[word].append(emotion)
        print(f"EmoLex cargado: {len(emolex)} palabras")
        return emolex
    except FileNotFoundError:
        print("Archivo EmoLex no encontrado")
        return {}

In [76]:
emolex = load_emolex('NRC-Emotion-Lexicon-Wordlevel-v0.92.txt')

EmoLex cargado: 6453 palabras


In [78]:
import csv
from collections import defaultdict

def load_emolex(filepath):
    # adapta si usas el fichero "full" o los ficheros por emoción
    word_emotions = defaultdict(set)  # word -> set(emotions)
    with open(filepath, encoding='utf-8') as f:
        reader = csv.reader(f, delimiter='\t')
        for row in reader:
            if len(row) < 3: continue
            word, emotion, association = row[0].strip(), row[1].strip(), row[2].strip()
            if association == '1':
                word_emotions[word].add(emotion)
    return word_emotions

In [80]:
emolex = load_emolex('NRC-Emotion-Lexicon-Wordlevel-v0.92.txt')

In [82]:
emolex

defaultdict(set,
            {'abacus': {'trust'},
             'abandon': {'fear', 'negative', 'sadness'},
             'abandoned': {'anger', 'fear', 'negative', 'sadness'},
             'abandonment': {'anger',
              'fear',
              'negative',
              'sadness',
              'surprise'},
             'abba': {'positive'},
             'abbot': {'trust'},
             'abduction': {'fear', 'negative', 'sadness', 'surprise'},
             'aberrant': {'negative'},
             'aberration': {'disgust', 'negative'},
             'abhor': {'anger', 'disgust', 'fear', 'negative'},
             'abhorrent': {'anger', 'disgust', 'fear', 'negative'},
             'ability': {'positive'},
             'abject': {'disgust', 'negative'},
             'abnormal': {'disgust', 'negative'},
             'abolish': {'anger', 'negative'},
             'abolition': {'negative'},
             'abominable': {'disgust', 'fear', 'negative'},
             'abomination': {'anger', 'di

#### Extensión del léxico usando WordNet

Vamos a extender el léxico NRC añadiendo sinónimos, hipónimos, hiperónimos y formas derivadas de las palabras originales. Usaremos WordNet y la función derivationally_related_forms().

Además, usaremos los diccionarios de mapeo de POS-tags proporcionados.

In [84]:
# Diccionarios para mapeo de POS-tags
wordnet_to_penn = {
    'n': 'NN',  # sustantivo
    'v': 'VB',  # verbo
    'a': 'JJ',  # adjetivo
    's': 'JJ',  # adjetivo superlativo
    'r': 'RB',  # adverbio
    'c': 'CC'   # conjunción
}

penn_to_wordnet = {
    'CC': 'c',   # Coordinating conjunction
    'CD': 'c',   # Cardinal number
    'DT': 'c',   # Determiner
    'EX': 'c',   # Existential there
    'FW': 'x',   # Foreign word
    'IN': 'c',   # Preposition or subordinating conjunction
    'JJ': 'a',   # Adjective
    'JJR': 'a',  # Adjective, comparative
    'JJS': 'a',  # Adjective, superlative
    'LS': 'c',   # List item marker
    'MD': 'v',   # Modal
    'NN': 'n',   # Noun, singular or mass
    'NNS': 'n',  # Noun, plural
    'NNP': 'n',  # Proper noun, singular
    'NNPS': 'n', # Proper noun, plural
    'PDT': 'c',  # Predeterminer
    'POS': 'c',  # Possessive ending
    'PRP': 'n',  # Personal pronoun
    'PRP$': 'n', # Possessive pronoun
    'RB': 'r',   # Adverb
    'RBR': 'r',  # Adverb, comparative
    'RBS': 'r',  # Adverb, superlative
    'RP': 'r',   # Particle
    'SYM': 'x',  # Symbol
    'TO': 'c',   # to
    'UH': 'x',   # Interjection
    'VB': 'v',   # Verb, base form
    'VBD': 'v',  # Verb, past tense
    'VBG': 'v',  # Verb, gerund or present participle
    'VBN': 'v',  # Verb, past participle
    'VBP': 'v',  # Verb, non-3rd person singular present
    'VBZ': 'v',  # Verb, 3rd person singular present
    'WDT': 'c',  # Wh-determiner
    'WP': 'n',   # Wh-pronoun
    'WP$': 'n',  # Possessive wh-pronoun
    'WRB': 'r',  # Wh-adverb
    'X': 'x'     # Any word not categorized by the other tags
}

# Inicializar el lematizador
lemmatizer = WordNetLemmatizer()

# Crear un diccionario extendido
extended_emolex = {}

# Función para obtener sinónimos, hipónimos, hiperónimos y formas derivadas
def get_wordnet_relations(word, pos_tag_wn):
    synsets = wn.synsets(word, pos=pos_tag_wn)
    relations = set()
    for synset in synsets:
        # Lemas del synset (sinónimos)
        for lemma in synset.lemmas():
            relations.add(lemma.name().replace('_', ' '))
        # Hipónimos
        for hypo in synset.hyponyms():
            for lemma in hypo.lemmas():
                relations.add(lemma.name().replace('_', ' '))
        # Hiperónimos
        for hyper in synset.hypernyms():
            for lemma in hyper.lemmas():
                relations.add(lemma.name().replace('_', ' '))
        # Formas derivadas (derivationally_related_forms)
        for lemma in synset.lemmas():
            for related_form in lemma.derivationally_related_forms():
                relations.add(related_form.name().replace('_', ' '))
    return relations

# Recorrer las palabras originales del EmoLex
for word, emotions in emolex.items():
    # Para cada palabra, intentar obtener su POS-tag usando WordNet
    # Inicialmente, no sabemos el POS-tag, así que probaremos con los cuatro principales: n, v, a, r
    for pos_wn in ['n', 'v', 'a', 'r']:
        # Obtener relaciones
        relations = get_wordnet_relations(word, pos_wn)
        # Para cada palabra relacionada, agregar las emociones
        for related_word in relations:
            # Convertir el POS-tag de WordNet a Penn
            pos_penn = wordnet_to_penn.get(pos_wn, None)
            if pos_penn:
                key = (related_word, pos_penn)
                if key not in extended_emolex:
                    extended_emolex[key] = set()
                extended_emolex[key].update(emotions)

    # También agregar la palabra original (sin POS-tag específico) pero necesitamos asignarle un POS-tag
    # Para la palabra original, usaremos el primer POS-tag que encontremos en WordNet
    synsets = wn.synsets(word)
    if synsets:
        pos_wn = synsets[0].pos()
        pos_penn = wordnet_to_penn.get(pos_wn, None)
        if pos_penn:
            key = (word, pos_penn)
            if key not in extended_emolex:
                extended_emolex[key] = set()
            extended_emolex[key].update(emotions)

# Convertir los sets a listas para consistencia
for key in extended_emolex:
    extended_emolex[key] = list(extended_emolex[key])

# Mostrar el tamaño del léxico extendido
print(f"Tamaño del léxico extendido: {len(extended_emolex)}")

Tamaño del léxico extendido: 61842


In [90]:
def extend_emolex(original_emolex):
    """
    Extiende el léxico EmoLex usando WordNet
    Retorna: dict con estructura {(lemma, pos_tag): [emociones]}
    """
    extended_lexicon = {}
    lemmatizer = WordNetLemmatizer()
    
    for word, emotions in original_emolex.items():
        # Probar diferentes POS tags para encontrar synsets
        for pos in ['n', 'v', 'a', 'r']:
            synsets = wn.synsets(word, pos=pos)
            
            for synset in synsets:
                # Añadir lema principal
                lemma = lemmatizer.lemmatize(word, pos=pos)
                pos_tag_penn = wordnet_to_penn.get(pos, 'NN')
                key = (lemma, pos_tag_penn)
                
                if key not in extended_lexicon:
                    extended_lexicon[key] = set()
                extended_lexicon[key].update(emotions)
                
                # Añadir sinónimos
                for lemma_obj in synset.lemmas():
                    synonym = lemma_obj.name().replace('_', ' ')
                    synonym_key = (synonym, pos_tag_penn)
                    if synonym_key not in extended_lexicon:
                        extended_lexicon[synonym_key] = set()
                    extended_lexicon[synonym_key].update(emotions)
                
                # Añadir hipónimos
                for hyponym in synset.hyponyms():
                    for lemma_obj in hyponym.lemmas():
                        hyponym_word = lemma_obj.name().replace('_', ' ')
                        hyponym_key = (hyponym_word, pos_tag_penn)
                        if hyponym_key not in extended_lexicon:
                            extended_lexicon[hyponym_key] = set()
                        extended_lexicon[hyponym_key].update(emotions)
                
                # Añadir hiperónimos
                for hypernym in synset.hypernyms():
                    for lemma_obj in hypernym.lemmas():
                        hypernym_word = lemma_obj.name().replace('_', ' ')
                        hypernym_key = (hypernym_word, pos_tag_penn)
                        if hypernym_key not in extended_lexicon:
                            extended_lexicon[hypernym_key] = set()
                        extended_lexicon[hypernym_key].update(emotions)
    
    # Convertir sets a listas
    return {key: list(emotions) for key, emotions in extended_lexicon.items()}

In [94]:
# Extender el léxico
extended_emolex = extend_emolex(emolex)
print(f"Léxico extendido: {len(extended_emolex)} entradas")
print("Ejemplo de entradas extendidas:", dict(list(extended_emolex.items())[:5]))

Léxico extendido: 51100 entradas
Ejemplo de entradas extendidas: {('abacus', 'NN'): ['trust', 'positive'], ('tablet', 'NN'): ['trust', 'positive'], ('calculator', 'NN'): ['disgust', 'trust', 'negative', 'fear', 'anger', 'positive', 'sadness'], ('calculating machine', 'NN'): ['disgust', 'trust', 'negative', 'fear', 'anger', 'positive', 'sadness'], ('abandon', 'NN'): ['negative', 'trust', 'fear', 'joy', 'anticipation', 'sadness', 'positive']}


#### Descarga de novelas de Project Gutenberg

Usamos la función download_text proporcionada para descargar las novelas.

In [96]:
# Diccionario de libros
books = {
    'Crime and Punishment': 'http://www.gutenberg.org/files/2554/2554-0.txt',
    'War and Peace': 'http://www.gutenberg.org/files/2600/2600-0.txt',
    'Pride and Prejudice': 'http://www.gutenberg.org/files/1342/1342-0.txt',
    'Frankenstein': 'https://www.gutenberg.org/cache/epub/84/pg84.txt',
    'The Adventures of Sherlock Holmes': 'http://www.gutenberg.org/files/1661/1661-0.txt',
    'Ulysses': 'http://www.gutenberg.org/files/4300/4300-0.txt',
    'The Odyssey': 'https://www.gutenberg.org/cache/epub/1727/pg1727.txt',
    'Moby Dick': 'http://www.gutenberg.org/files/15/15-0.txt',
    'The Divine Comedy': 'https://www.gutenberg.org/cache/epub/8800/pg8800.txt',
    'Critias': 'https://www.gutenberg.org/cache/epub/1571/pg1571.txt'
}

def download_text(url):
    """Descarga el texto de una novela desde Project Gutenberg"""
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"Error al descargar: {e}")
        return None
        
# Descargar todos los libros
book_texts = {}
for title, url in books.items():
    print(f"Descargando {title}...")
    text = download_text(url)
    if text:
        book_texts[title] = text
        print(f"{title} descargado correctamente.")
    else:
        print(f"Fallo en la descarga de {title}.")

Descargando Crime and Punishment...
Crime and Punishment descargado correctamente.
Descargando War and Peace...
War and Peace descargado correctamente.
Descargando Pride and Prejudice...
Pride and Prejudice descargado correctamente.
Descargando Frankenstein...
Frankenstein descargado correctamente.
Descargando The Adventures of Sherlock Holmes...
The Adventures of Sherlock Holmes descargado correctamente.
Descargando Ulysses...
Ulysses descargado correctamente.
Descargando The Odyssey...
The Odyssey descargado correctamente.
Descargando Moby Dick...
Moby Dick descargado correctamente.
Descargando The Divine Comedy...
The Divine Comedy descargado correctamente.
Descargando Critias...
Critias descargado correctamente.


#### Análisis de texto

Implementamos la función para analizar el texto y contar las emociones.

In [102]:
# Inicializar el lematizador
lemmatizer = WordNetLemmatizer()

def analyze_emotions(text, lexicon):
    # Tokenización
    tokens = word_tokenize(text)
    # POS-tagging
    pos_tags = pos_tag(tokens)
    # Inicializar contador de emociones
    emotion_counts = {}
    # Procesar cada token
    for word, pos_tag_pen in pos_tags:
        # Convertir POS-tag de PennTreeBank a WordNet
        pos_wn = penn_to_wordnet.get(pos_tag_pen, None)
        if pos_wn is None:
            continue
        # Lematización
        lemma = lemmatizer.lemmatize(word, pos=pos_wn)
        # Buscar en el léxico extendido
        key = (lemma, pos_tag_pen)
        if key in lexicon:
            emotions = lexicon[key]
            for emotion in emotions:
                emotion_counts[emotion] = emotion_counts.get(emotion, 0) + 1
    return emotion_counts

# Analizar cada libro
results = {}
for title, text in book_texts.items():
    print(f"Analizando {title}...")
    results[title] = analyze_emotions(text, extended_emolex)

Analizando Crime and Punishment...


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\Tania/nltk_data'
    - 'C:\\Users\\Tania\\anaconda3\\nltk_data'
    - 'C:\\Users\\Tania\\anaconda3\\share\\nltk_data'
    - 'C:\\Users\\Tania\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\Tania\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


#### Presentación de resultados

Vamos a mostrar los resultados en forma de tabla y gráficos.

In [66]:
# Crear un DataFrame con los resultados
df = pd.DataFrame.from_dict(results, orient='index')
# Rellenar NaN con 0
df = df.fillna(0)
# Mostrar la tabla
df

# Graficar los resultados
plt.figure(figsize=(12, 8))
df.plot(kind='bar', stacked=True, figsize=(12,8))
plt.title('Emociones en novelas clásicas')
plt.ylabel('Frecuencia')
plt.xlabel('Novelas')
plt.xticks(rotation=45)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

TypeError: no numeric data to plot

<Figure size 1200x800 with 0 Axes>

#### Conclusiones

En esta sección, comenta brevemente los resultados obtenidos.
* ¿Qué emociones son las más frecuentes en general?
* ¿Hay novelas que destacan por alguna emoción en particular?
* ¿Qué limitaciones has encontrado?

----

# Introducción 
En esta práctica realizaremos un análisis de emociones en textos literarios clásicos disponibles en Project Gutenberg. Utilizaremos el léxico NRC EmoLex extendido con WordNet para identificar y contar emociones expresadas en las obras.

## Instalación y carga de bibliotecas

In [3]:
# Instalación de dependencias (ejecutar si es necesario)
# !pip install nltk requests matplotlib seaborn pandas

In [1]:
import nltk
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter, defaultdict
from nltk.corpus import wordnet as wn
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

# Descargar recursos necesarios de NLTK
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)
nltk.download('punkt_tab', quiet=True)   ## Añadido por dani para ¿borrar chunk de abajo?

print("Bibliotecas cargadas correctamente")

Bibliotecas cargadas correctamente


In [ ]:
## BORRAR ¿?
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')  # <-- este es el que falta
nltk.download('averaged_perceptron_tagger')  # para POS tagging
nltk.download('wordnet')  # para lematización
nltk.download('omw-1.4')  # para soporte multilingüe de WordNet

In [ ]:
## BORRAR ¿?

import nltk

resources = ['punkt', 'punkt_tab', 'averaged_perceptron_tagger', 'wordnet', 'omw-1.4']

for resource in resources:
    try:
        nltk.data.find(resource)
    except LookupError:
        nltk.download(resource)

## Tarea 1: Cargar el léxico NRC EmoLex (1.5 puntos)

Cargamos el Word-Emotion Association Lexicon del NRC, que contiene 14,182 palabras asociadas con 8 emociones y 2 sentimientos.

In [ ]:
# Este sí que lo pondría como código (osea, no como función)
# y podríamos añadir lo de borrar aquellos que sean todo 0 (sin asociación con emoción)

# Si quieres puedo copiar el código abajo y lo pruebo (incluyendo la parte de la ampliación del diccionario)
# de ese modo podemos comparar el tiempo y las dimensiones de los diccionarios.

def load_nrc_lexicon_from_file(filepath):
    """
    Carga el léxico NRC desde el archivo oficial.
    
    Args:
        filepath: Ruta al archivo NRC-Emotion-Lexicon-Wordlevel-v0.92.txt
    
    Returns:
        dict: Diccionario {palabra: [lista_de_emociones]}
    """
    nrc_lexicon = defaultdict(list)
    
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            for line in f:
                parts = line.strip().split('\t')
                if len(parts) == 3:
                    word, emotion, association = parts
                    if association == '1':
                        nrc_lexicon[word].append(emotion)
        
        print(f"Léxico NRC cargado desde archivo con {len(nrc_lexicon)} palabras")
        return dict(nrc_lexicon)
    except FileNotFoundError:
        print(f"Archivo no encontrado: {filepath}")
        return load_nrc_lexicon()

In [20]:
# Cargar el léxico
nrc_lexicon = load_nrc_lexicon_from_file('NRC-Emotion-Lexicon-Wordlevel-v0.92.txt')

Léxico NRC cargado desde archivo con 6453 palabras


In [2]:
# Versión Dani

# Carga el léxico NRC desde el archivo oficial: NRC-Emotion-Lexicon-Wordlevel-v0.92.txt
# Nos devolverá un Diccionario {palabra: [lista_de_emociones]}

archivo = 'NRC-Emotion-Lexicon-Wordlevel-v0.92.txt'

# Preparamos el diccionario
nrc_lexicon = defaultdict(list) 

try:
    with open(archivo, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split('\t')
            if len(parts) == 3:
                word, emotion, association = parts
                if association == '1':
                    nrc_lexicon[word].append(emotion)  # De esta manera almacenamos las emociones en las que sí existe asociación
    
    print(f"Léxico NRC cargado desde archivo con {len(nrc_lexicon)} palabras")

except FileNotFoundError:
    print(f"Archivo no encontrado: {archivo}")

Léxico NRC cargado desde archivo con 6453 palabras


In [3]:
# Mostrar ejemplo
print("\nEjemplo de entradas del léxico:")
for word in list(nrc_lexicon.keys())[:5]:
    print(f"  {word}: {nrc_lexicon[word]}")


Ejemplo de entradas del léxico:
  abacus: ['trust']
  abandon: ['fear', 'negative', 'sadness']
  abandoned: ['anger', 'fear', 'negative', 'sadness']
  abandonment: ['anger', 'fear', 'negative', 'sadness', 'surprise']
  abba: ['positive']


## Tarea 2: Extender EmoLex con WordNet (3.0 puntos)

Extenderemos el léxico usando sinónimos, hipónimos, hiperónimos y formas derivadas de WordNet. El léxico extendido usará claves (lemma, POS-tag).

In [5]:
# Diccionarios de conversión entre POS tags
wordnet_to_penn = {
    'n': 'NN',  # sustantivo
    'v': 'VB',  # verbo
    'a': 'JJ',  # adjetivo
    's': 'JJS', # adjetivo superlativo (le añadimos la "S") y cambiamos la 'a' por 's'
    'r': 'RB',  # adverbio
    'c': 'CC'   # conjunción
}

penn_to_wordnet = {
    'CC': 'c', 'CD': 'c', 'DT': 'c', 'EX': 'c', 'FW': 'x', 'IN': 'c',
    'JJ': 'a', 'JJR': 'a', 'JJS': 's', 'LS': 'c', 'MD': 'v',            # Cambiamos el 'JJS': 'a' por 'JJS': 's'
    'NN': 'n', 'NNS': 'n', 'NNP': 'n', 'NNPS': 'n',
    'PDT': 'c', 'POS': 'c', 'PRP': 'n', 'PRP$': 'n',
    'RB': 'r', 'RBR': 'r', 'RBS': 'r', 'RP': 'r',
    'SYM': 'x', 'TO': 'c', 'UH': 'x',
    'VB': 'v', 'VBD': 'v', 'VBG': 'v', 'VBN': 'v', 'VBP': 'v', 'VBZ': 'v',
    'WDT': 'c', 'WP': 'n', 'WP$': 'n', 'WRB': 'r', 'X': 'x'
}

In [22]:
# Esta quizás también la podemos poner como código ya que sólo se ejecutaría una vez para ampliar el diccionario

def extend_lexicon_with_wordnet(nrc_lexicon):
    """
    Extiende el léxico NRC usando relaciones de WordNet.
    
    Para cada palabra en el léxico original:
    - Obtiene sinónimos (synsets)
    - Obtiene hipónimos (términos más específicos)
    - Obtiene hiperónimos (términos más generales)
    - Obtiene formas derivadas (como plurales)
    
    Args:
        nrc_lexicon: Diccionario {palabra: [emociones]}
    
    Returns:
        dict: Diccionario {(lemma, pos_tag): [emociones]}
    """
    extended_lexicon = {}
    
    print("Extendiendo léxico con WordNet...")
    
    for word, emotions in nrc_lexicon.items():
        # Obtener synsets de la palabra
        synsets = wn.synsets(word)
        
        for synset in synsets:
            # POS tag de WordNet
            wn_pos = synset.pos()
            penn_pos = wordnet_to_penn.get(wn_pos, 'NN')
            
            # 1. Agregar la palabra original
            extended_lexicon[(word, penn_pos)] = emotions
            
            # 2. Agregar sinónimos del synset
            for lemma in synset.lemmas():
                lemma_name = lemma.name().replace('_', ' ').lower()
                extended_lexicon[(lemma_name, penn_pos)] = emotions
                
                # 3. Agregar formas derivadas
                try:
                    for related in lemma.derivationally_related_forms():
                        related_pos = related.synset().pos()
                        related_penn = wordnet_to_penn.get(related_pos, 'NN')
                        related_name = related.name().replace('_', ' ').lower()
                        extended_lexicon[(related_name, related_penn)] = emotions
                except:
                    pass
            
            # 4. Agregar hipónimos (términos más específicos)
            for hyponym in synset.hyponyms():
                for lemma in hyponym.lemmas():
                    hyp_name = lemma.name().replace('_', ' ').lower()
                    hyp_pos = wordnet_to_penn.get(hyponym.pos(), 'NN')
                    extended_lexicon[(hyp_name, hyp_pos)] = emotions
            
            # 5. Agregar hiperónimos (términos más generales)
            for hypernym in synset.hypernyms():
                for lemma in hypernym.lemmas():
                    hyper_name = lemma.name().replace('_', ' ').lower()
                    hyper_pos = wordnet_to_penn.get(hypernym.pos(), 'NN')
                    extended_lexicon[(hyper_name, hyper_pos)] = emotions
    
    print(f"Léxico extendido de {len(nrc_lexicon)} a {len(extended_lexicon)} entradas")
    
    return extended_lexicon

In [24]:
# Extender el léxico
extended_lexicon2 = extend_lexicon_with_wordnet(nrc_lexicon)

Extendiendo léxico con WordNet...
Léxico extendido de 6453 a 55155 entradas


In [21]:
## BORRAR
lista=['abandon', 'abba']
#synset = wn.synsets(lista)  
synsets = wn.synsets('abandon')
print(synsets) 

for synset in synsets:
    print(synset) 
    wn_pos = synset.pos()
    print(wn_pos)
    penn_pos = wordnet_to_penn.get(wn_pos, 'NN')
    print(penn_pos)
    penn_pos2 = wordnet_to_penn.get(wn_pos)
    print(penn_pos2)


dict1 = {'nombre': 'John', 'edad': 4, 'puntaje': 45} # diccionario
print(len(dict1))
dict2 = {'nombre': 'John', 'edad': 4, 'puntaje': [45, 17]} # diccionario
print(len(dict2))

[Synset('abandon.n.01'), Synset('wildness.n.01'), Synset('abandon.v.01'), Synset('abandon.v.02'), Synset('vacate.v.02'), Synset('abandon.v.04'), Synset('abandon.v.05')]
Synset('abandon.n.01')
n
NN
NN
Synset('wildness.n.01')
n
NN
NN
Synset('abandon.v.01')
v
VB
VB
Synset('abandon.v.02')
v
VB
VB
Synset('vacate.v.02')
v
VB
VB
Synset('abandon.v.04')
v
VB
VB
Synset('abandon.v.05')
v
VB
VB
3
3


In [ ]:
# Versión de Dani


# Extiende el léxico NRC usando relaciones de WordNet y el diccionario generado en el apartado 1: Diccionario {palabra: [emociones]}.

# Para cada palabra en el léxico original:
# - Obtiene sinónimos (synsets)
# - Obtiene hipónimos (términos más específicos)
# - Obtiene hiperónimos (términos más generales)
# - Obtiene formas derivadas (como plurales)

# Obtendremos un diccionario {(lemma, pos_tag): [emociones]} con dichas impletmentaciones

extended_lexicon = {}

print("Extendiendo léxico con WordNet...")

for word, emotions in nrc_lexicon.items():
    # Obtener synsets de la palabra
    synsets = wn.synsets(word)
    
    for synset in synsets:
        # POS tag de WordNet y buscamos su complementario en penn
        wn_pos = synset.pos()
        penn_pos = wordnet_to_penn.get(wn_pos, 'NN')    # Le indicamos que, en caso de no encontrar su correspondiente en el diccionario utilice 'NN'
        
        # Agregamos la palabra original
        key = (word, penn_pos)    ##### ¿Esto es correcto? ¿Es ese word un lemma?
        if key in extended_lexicon:
            # Para combinar sin duplicar y no arriesgarnos a perder información
            extended_lexicon[key] = list(set(extended_lexicon[key]) | set(emotions))
        else:
            extended_lexicon[key] = emotions
        
        # Agregamos los sinónimos del synset
        for lemma in synset.lemmas():
            lemma_name = lemma.name().replace('_', ' ').lower()
            key = (lemma_name, penn_pos)
            if key in extended_lexicon:
                # Para combinar sin duplicar y no arriesgarnos a perder información
                extended_lexicon[key] = list(set(extended_lexicon[key]) | set(emotions))
            else:
                extended_lexicon[key] = emotions
            
            
            # 3. Agregamos formas derivadas
            try:
                for related in lemma.derivationally_related_forms():
                    related_pos = related.synset().pos()
                    related_penn = wordnet_to_penn.get(related_pos, 'NN')
                    related_name = related.name().replace('_', ' ').lower()
                    key = (related_name, related_penn)
                    if key in extended_lexicon:
                        # Para combinar sin duplicar y no arriesgarnos a perder información
                        extended_lexicon[key] = list(set(extended_lexicon[key]) | set(emotions))
                    else:
                        extended_lexicon[key] = emotions
            except:
                pass
        
        # Agregamos los hipónimos (términos más específicos)
        for hyponym in synset.hyponyms():
            for lemma in hyponym.lemmas():
                hyp_name = lemma.name().replace('_', ' ').lower()
                hyp_pos = wordnet_to_penn.get(hyponym.pos(), 'NN')
                key = (hyp_name, hyp_pos)
                if key in extended_lexicon:
                    # Para combinar sin duplicar y no arriesgarnos a perder información
                    extended_lexicon[key] = list(set(extended_lexicon[key]) | set(emotions))
                else:
                    extended_lexicon[key] = emotions
        
        # Agragamos los hiperónimos (términos más generales)
        for hypernym in synset.hypernyms():
            for lemma in hypernym.lemmas():
                hyper_name = lemma.name().replace('_', ' ').lower()
                hyper_pos = wordnet_to_penn.get(hypernym.pos(), 'NN')
                key = (hyper_name, hyper_pos)
                if key in extended_lexicon:
                    # Para combinar sin duplicar y no arriesgarnos a perder información
                    extended_lexicon[key] = list(set(extended_lexicon[key]) | set(emotions))
                else:
                    extended_lexicon[key] = emotions

# Nos aseguramos de que no haya duplicados y de que estén ordenados
for k in extended_lexicon:
    extended_lexicon[k] = sorted(set(extended_lexicon[k]))

print(f"Léxico extendido de {len(nrc_lexicon)} a {len(extended_lexicon)} entradas")


Extendiendo léxico con WordNet...
Léxico extendido de 6453 a 55155 entradas


In [ ]:
# Versión de Dani versión para convertir en lemma


# Extiende el léxico NRC usando relaciones de WordNet y el diccionario generado en el apartado 1: Diccionario {palabra: [emociones]}.

# Para cada palabra en el léxico original:
# - Obtiene sinónimos (synsets)
# - Obtiene hipónimos (términos más específicos)
# - Obtiene hiperónimos (términos más generales)
# - Obtiene formas derivadas (como plurales)

# Obtendremos un diccionario {(lemma, pos_tag): [emociones]} con dichas impletmentaciones

extended_lexicon = {}

print("Extendiendo léxico con WordNet...")

for word, emotions in nrc_lexicon.items():
    # Obtener synsets de la palabra
    synsets = wn.synsets(word)
    
    for synset in synsets:
        # POS tag de WordNet y buscamos su complementario en penn
        wn_pos = synset.pos()
        penn_pos = wordnet_to_penn.get(wn_pos, 'NN')    # Le indicamos que, en caso de no encontrar su correspondiente en el diccionario utilice 'NN'
        
        # Agregamos la palabra original
        key = (word, penn_pos)    ##### ¿Esto es correcto? ¿Es ese word un lemma?
        if key in extended_lexicon:
            # Para combinar sin duplicar y no arriesgarnos a perder información
            extended_lexicon[key] = list(set(extended_lexicon[key]) | set(emotions))
        else:
            extended_lexicon[key] = emotions
        
        # Agregamos los sinónimos del synset
        for lemma in synset.lemmas():
            lemma_name = lemma.name().replace('_', ' ').lower()
            key = (lemma_name, penn_pos)
            if key in extended_lexicon:
                # Para combinar sin duplicar y no arriesgarnos a perder información
                extended_lexicon[key] = list(set(extended_lexicon[key]) | set(emotions))
            else:
                extended_lexicon[key] = emotions
            
            
            # 3. Agregamos formas derivadas
            try:
                for related in lemma.derivationally_related_forms():
                    related_pos = related.synset().pos()
                    related_penn = wordnet_to_penn.get(related_pos, 'NN')
                    related_name = related.name().replace('_', ' ').lower()
                    key = (related_name, related_penn)
                    if key in extended_lexicon:
                        # Para combinar sin duplicar y no arriesgarnos a perder información
                        extended_lexicon[key] = list(set(extended_lexicon[key]) | set(emotions))
                    else:
                        extended_lexicon[key] = emotions
            except:
                pass
        
        # Agregamos los hipónimos (términos más específicos)
        for hyponym in synset.hyponyms():
            for lemma in hyponym.lemmas():
                hyp_name = lemma.name().replace('_', ' ').lower()
                hyp_pos = wordnet_to_penn.get(hyponym.pos(), 'NN')
                key = (hyp_name, hyp_pos)
                if key in extended_lexicon:
                    # Para combinar sin duplicar y no arriesgarnos a perder información
                    extended_lexicon[key] = list(set(extended_lexicon[key]) | set(emotions))
                else:
                    extended_lexicon[key] = emotions
        
        # Agragamos los hiperónimos (términos más generales)
        for hypernym in synset.hypernyms():
            for lemma in hypernym.lemmas():
                hyper_name = lemma.name().replace('_', ' ').lower()
                hyper_pos = wordnet_to_penn.get(hypernym.pos(), 'NN')
                key = (hyper_name, hyper_pos)
                if key in extended_lexicon:
                    # Para combinar sin duplicar y no arriesgarnos a perder información
                    extended_lexicon[key] = list(set(extended_lexicon[key]) | set(emotions))
                else:
                    extended_lexicon[key] = emotions

# Nos aseguramos de que no haya duplicados y de que estén ordenados
for k in extended_lexicon:
    extended_lexicon[k] = sorted(set(extended_lexicon[k]))

print(f"Léxico extendido de {len(nrc_lexicon)} a {len(extended_lexicon)} entradas")


Extendiendo léxico con WordNet...
Léxico extendido de 6453 a 55155 entradas


In [ ]:
## BORRAR

# El código de arriba se puede mejorar con esta función `para evitar hacer lo mismo todo el rato
def add_to_lexicon(key, emotions):
    if key in extended_lexicon:
        extended_lexicon[key] = list(set(extended_lexicon[key]) | set(emotions))
    else:
        extended_lexicon[key] = emotions
    
# así se llama
add_to_lexicon(key, emotions)

In [ ]:
# Extender el léxico
extended_lexicon = extend_lexicon_with_wordnet(nrc_lexicon)

Extendiendo léxico con WordNet...
Léxico extendido de 6453 a 54349 entradas


In [ ]:
# Mostrar ejemplos
print("\nEjemplos del léxico extendido:")
for key in list(extended_lexicon.keys())[:10]:
    print(f"  {key}: {extended_lexicon[key]}")

## BORRAR ¿?
print("\nEjemplos del léxico extendido sin conservar emociones:")
for key2 in list(extended_lexicon2.keys())[:10]:
    print(f"  {key2}: {extended_lexicon2[key2]}")


Ejemplos del léxico extendido:
  ('abacus', 'NN'): ['positive', 'trust']
  ('tablet', 'NN'): ['positive', 'trust']
  ('calculator', 'NN'): ['anger', 'disgust', 'fear', 'negative', 'positive', 'sadness', 'trust']
  ('calculating machine', 'NN'): ['anger', 'disgust', 'fear', 'negative', 'positive', 'sadness', 'trust']
  ('abandon', 'NN'): ['anticipation', 'fear', 'joy', 'negative', 'positive', 'sadness', 'trust']
  ('wantonness', 'NN'): ['anger', 'disgust', 'fear', 'negative', 'sadness']
  ('wanton', 'JJS'): ['fear', 'negative', 'sadness']
  ('unconstraint', 'NN'): ['fear', 'negative', 'sadness']
  ('unrestraint', 'NN'): ['fear', 'negative', 'sadness']
  ('wildness', 'NN'): ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'negative', 'positive', 'sadness', 'surprise', 'trust']

Ejemplos del léxico extendido sin conservar emociones:
  ('abacus', 'NN'): ['positive', 'trust']
  ('tablet', 'NN'): ['positive']
  ('calculator', 'NN'): ['trust']
  ('calculating machine', 'NN'): ['trust']
  

## Tarea 3: Cargar novelas de Project Gutenberg (1.5 puntos)

Descargaremos 10 novelas clásicas desde Project Gutenberg.

In [40]:
books = {
    'Crime and Punishment': 'http://www.gutenberg.org/files/2554/2554-0.txt',
    'War and Peace': 'http://www.gutenberg.org/files/2600/2600-0.txt',
    'Pride and Prejudice': 'http://www.gutenberg.org/files/1342/1342-0.txt',
    'Frankenstein': 'https://www.gutenberg.org/cache/epub/84/pg84.txt',
    'The Adventures of Sherlock Holmes': 'http://www.gutenberg.org/files/1661/1661-0.txt',
    'Ulysses': 'http://www.gutenberg.org/files/4300/4300-0.txt',
    'The Odyssey': 'https://www.gutenberg.org/cache/epub/1727/pg1727.txt',
    'Moby Dick': 'http://www.gutenberg.org/files/15/15-0.txt',
    'The Divine Comedy': 'https://www.gutenberg.org/cache/epub/8800/pg8800.txt',
    'Critias': 'https://www.gutenberg.org/cache/epub/1571/pg1571.txt'
}

In [ ]:
def download_text(url):
    """
    Descarga el texto de una novela en formato txt.
    
    Args:
        url: URL del texto en Project Gutenberg
    
    Returns:
        str: Contenido del texto o None si hay error
    """
    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"Error al descargar el texto: {e}")
        return None

## ¿Esta función quita también los saltos de página?
## ¿Habría que eliminar los símbolos de puntuación por si hacen que haya palabras no reconocibles?
def clean_gutenberg_text(text):
    """
    Limpia el texto eliminando el header y footer de Project Gutenberg.
    
    Args:
        text: Texto completo del libro
    
    Returns:
        str: Texto limpio
    """
    # Buscar inicio y fin del contenido real
    start_markers = ['*** START OF THIS PROJECT GUTENBERG', '*** START OF THE PROJECT GUTENBERG']
    end_markers = ['*** END OF THIS PROJECT GUTENBERG', '*** END OF THE PROJECT GUTENBERG']
    
    start_idx = 0
    for marker in start_markers:
        idx = text.find(marker)
        if idx != -1:
            start_idx = text.find('\n', idx) + 1
            break
    
    end_idx = len(text)
    for marker in end_markers:
        idx = text.find(marker)
        if idx != -1:
            end_idx = idx
            break
    
    return text[start_idx:end_idx].strip()


In [ ]:
## cambiamos o dejamos los simbolos (ticks y cruces)
# Descargar todas las novelas
book_texts = {}

print("Descargando novelas de Project Gutenberg...\n")

for title, url in books.items():
    print(f"Descargando: {title}...", end=' ')
    text = download_text(url)
    
    if text:
        cleaned_text = clean_gutenberg_text(text)
        book_texts[title] = cleaned_text
        print(f"✓ ({len(cleaned_text):,} caracteres)")
    else:
        print("✗ Error")

print(f"\n✓ {len(book_texts)} novelas descargadas correctamente")


Descargando novelas de Project Gutenberg...

Descargando: Crime and Punishment... ✓ (1,135,108 caracteres)
Descargando: War and Peace... ✓ (3,273,921 caracteres)
Descargando: Pride and Prejudice... ✓ (743,241 caracteres)
Descargando: Frankenstein... ✓ (426,692 caracteres)
Descargando: The Adventures of Sherlock Holmes... ✓ (574,143 caracteres)
Descargando: Ulysses... ✓ (1,552,547 caracteres)
Descargando: The Odyssey... ✓ (690,677 caracteres)
Descargando: Moby Dick... ✓ (1,261,116 caracteres)
Descargando: The Divine Comedy... ✓ (620,276 caracteres)
Descargando: Critias... ✓ (55,996 caracteres)

✓ 10 novelas descargadas correctamente


## Tarea 4: Analizar emociones en los textos (3.0 puntos)

Implementaremos un analizador que tokenice, haga POS-tagging, lematice y cuente las emociones en cada novela.

In [87]:
def get_wordnet_pos(penn_tag):
    """
    Convierte un POS tag de Penn Treebank a formato WordNet.
    
    Args:
        penn_tag: Tag de Penn Treebank
    
    Returns:
        str: Tag de WordNet
    """
    return penn_to_wordnet.get(penn_tag, 'n')

def analyze_emotions(text, extended_lexicon):
    """
    Analiza las emociones presentes en un texto.
    
    Proceso:
    1. Tokenización: divide el texto en palabras
    2. POS-tagging: identifica la categoría gramatical de cada palabra
    3. Lematización: reduce cada palabra a su forma base
    4. Búsqueda en léxico: compara (lemma, POS) con el léxico extendido
    5. Conteo: cuenta las ocurrencias de cada emoción
    
    Args:
        text: Texto a analizar
        extended_lexicon: Diccionario {(lemma, pos): [emociones]}
    
    Returns:
        Counter: Contador de emociones encontradas
    """
    # Inicializar el lematizador
    lemmatizer = WordNetLemmatizer()
    
    # Tokenizar el texto
    tokens = word_tokenize(text.lower())
    
    # Realizar POS-tagging
    pos_tags = pos_tag(tokens)
    
    # Contador de emociones
    emotion_counts = Counter()
    
    # Analizar cada palabra
    for word, penn_pos in pos_tags:
        # Solo procesar palabras alfabéticas
        if not word.isalpha():
            continue
        
        # Convertir POS tag a formato WordNet
        wn_pos = get_wordnet_pos(penn_pos)
        
        # Lematizar la palabra
        lemma = lemmatizer.lemmatize(word, pos=wn_pos)
        
        # Buscar en el léxico extendido
        key = (lemma, penn_pos)
        
        if key in extended_lexicon:
            emotions = extended_lexicon[key]
            for emotion in emotions:
                emotion_counts[emotion] += 1
    
    return emotion_counts

def analyze_all_books(book_texts, extended_lexicon):
    """
    Analiza todas las novelas y genera estadísticas.
    
    Args:
        book_texts: Diccionario {título: texto}
        extended_lexicon: Diccionario del léxico extendido
    
    Returns:
        dict: Resultados del análisis por libro
    """
    results = {}
    
    print("Analizando emociones en las novelas...\n")
    
    for title, text in book_texts.items():
        print(f"Analizando: {title}...", end=' ')
        
        # Limitar el texto para procesamiento más rápido (opcional)
        # En producción, procesar el texto completo
        text_sample = text[:100000]  # Primeros 100k caracteres
        
        emotion_counts = analyze_emotions(text_sample, extended_lexicon)
        results[title] = emotion_counts
        
        total_emotions = sum(emotion_counts.values())
        print(f"({total_emotions} emociones detectadas)")
    
    print(f"\nAnálisis completado para {len(results)} novelas")
    
    return results


In [91]:
# Realizar el análisis
analysis_results = analyze_all_books(book_texts, extended_lexicon)

Analizando emociones en las novelas...

Analizando: Crime and Punishment... 

LookupError: 
**********************************************************************
  Resource [93maveraged_perceptron_tagger_eng[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('averaged_perceptron_tagger_eng')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtaggers/averaged_perceptron_tagger_eng/[0m

  Searched in:
    - 'C:\\Users\\Tania/nltk_data'
    - 'C:\\Users\\Tania\\anaconda3\\nltk_data'
    - 'C:\\Users\\Tania\\anaconda3\\share\\nltk_data'
    - 'C:\\Users\\Tania\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\Tania\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


## Tarea 5: Presentar resultados (1.0 puntos)

Visualizaremos y analizaremos los patrones emocionales encontrados en las novelas.

In [ ]:
def create_results_dataframe(analysis_results):
    """
    Crea un DataFrame con los resultados del análisis.
    
    Args:
        analysis_results: Diccionario con resultados por libro
    
    Returns:
        DataFrame: Datos organizados para visualización
    """
    data = []
    
    for book, emotions in analysis_results.items():
        for emotion, count in emotions.items():
            data.append({
                'Book': book,
                'Emotion': emotion,
                'Count': count
            })
    
    return pd.DataFrame(data)

In [ ]:
# Crear DataFrame
df_results = create_results_dataframe(analysis_results)

# Mostrar resumen estadístico
print("=== RESUMEN ESTADÍSTICO ===\n")

# Total de emociones por libro
print("Total de emociones detectadas por libro:")
for book, emotions in analysis_results.items():
    total = sum(emotions.values())
    print(f"  {book}: {total:,}")

print("\n" + "="*50 + "\n")

# Emociones más comunes globalmente
print("Emociones más comunes en todas las novelas:")
all_emotions = Counter()
for emotions in analysis_results.values():
    all_emotions.update(emotions)

for emotion, count in all_emotions.most_common(10):
    print(f"  {emotion}: {count:,}")

### Visualización 1: Emociones por libro

In [ ]:
# Configurar estilo de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Crear figura con subplots
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Análisis de Emociones en Textos Literarios Clásicos', fontsize=16, fontweight='bold')

# 1. Heatmap de emociones por libro
ax1 = axes[0, 0]
pivot_table = df_results.pivot_table(values='Count', index='Book', columns='Emotion', fill_value=0)
sns.heatmap(pivot_table, annot=False, cmap='YlOrRd', ax=ax1, cbar_kws={'label': 'Frecuencia'})
ax1.set_title('Distribución de Emociones por Novela (Heatmap)', fontweight='bold')
ax1.set_xlabel('Emoción')
ax1.set_ylabel('Novela')
plt.setp(ax1.get_xticklabels(), rotation=45, ha='right')
plt.setp(ax1.get_yticklabels(), rotation=0)

# 2. Top 5 libros por total de emociones
ax2 = axes[0, 1]
book_totals = df_results.groupby('Book')['Count'].sum().sort_values(ascending=False).head(5)
book_totals.plot(kind='barh', ax=ax2, color='steelblue')
ax2.set_title('Top 5 Novelas con Más Expresiones Emocionales', fontweight='bold')
ax2.set_xlabel('Frecuencia Total')
ax2.set_ylabel('')

# 3. Distribución global de emociones
ax3 = axes[1, 0]
emotion_totals = df_results.groupby('Emotion')['Count'].sum().sort_values(ascending=False)
emotion_totals.plot(kind='bar', ax=ax3, color='coral')
ax3.set_title('Distribución Global de Emociones', fontweight='bold')
ax3.set_xlabel('Emoción')
ax3.set_ylabel('Frecuencia Total')
plt.setp(ax3.get_xticklabels(), rotation=45, ha='right')

# 4. Emociones por categoría (positivas vs negativas)
ax4 = axes[1, 1]
sentiment_data = df_results[df_results['Emotion'].isin(['positive', 'negative'])]
if not sentiment_data.empty:
    sentiment_totals = sentiment_data.groupby('Emotion')['Count'].sum()
    ax4.pie(sentiment_totals, labels=sentiment_totals.index, autopct='%1.1f%%', 
            colors=['lightgreen', 'lightcoral'], startangle=90)
    ax4.set_title('Proporción de Sentimientos (Positivo vs Negativo)', fontweight='bold')
else:
    ax4.text(0.5, 0.5, 'No hay datos de sentimiento', ha='center', va='center')
    ax4.set_title('Proporción de Sentimientos', fontweight='bold')

plt.tight_layout()
plt.show()


### Visualización 2: Análisis detallado por novela

In [ ]:
# Crear gráficos individuales para cada novela (top 5)
top_books = df_results.groupby('Book')['Count'].sum().sort_values(ascending=False).head(5).index

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('Perfil Emocional Detallado de las Principales Novelas', fontsize=16, fontweight='bold')

for idx, book in enumerate(top_books):
    row = idx // 3
    col = idx % 3
    ax = axes[row, col]
    
    book_data = df_results[df_results['Book'] == book].sort_values('Count', ascending=False).head(8)
    
    ax.bar(range(len(book_data)), book_data['Count'], color='teal', alpha=0.7)
    ax.set_xticks(range(len(book_data)))
    ax.set_xticklabels(book_data['Emotion'], rotation=45, ha='right')
    ax.set_title(book, fontweight='bold', fontsize=10)
    ax.set_ylabel('Frecuencia')
    ax.grid(axis='y', alpha=0.3)

# Ocultar el último subplot si no se usa
if len(top_books) < 6:
    axes[1, 2].axis('off')

plt.tight_layout()
plt.show()


## Análisis y Conclusiones

In [ ]:
print("="*70)
print(" ANÁLISIS DE PATRONES EMOCIONALES ENCONTRADOS")
print("="*70)

# Analizar qué libro es más emocional
most_emotional_book = max(analysis_results.items(), key=lambda x: sum(x[1].values()))
print(f"\nNovela más expresiva emocionalmente:")
print(f"   → {most_emotional_book[0]} ({sum(most_emotional_book[1].values()):,} expresiones)")

# Analizar la emoción dominante por libro
print(f"\nEmoción dominante por novela:")
for book, emotions in analysis_results.items():
    if emotions:
        dominant_emotion = max(emotions.items(), key=lambda x: x[1])
        print(f"   → {book}: {dominant_emotion[0]} ({dominant_emotion[1]:,})")

# Comparar sentimientos positivos vs negativos
print(f"\nBalance emocional (positivo vs negativo):")
for book, emotions in analysis_results.items():
    positive = emotions.get('positive', 0)
    negative = emotions.get('negative', 0)
    total = positive + negative
    if total > 0:
        pos_ratio = (positive / total) * 100
        print(f"   → {book}: {pos_ratio:.1f}% positivo, {100-pos_ratio:.1f}% negativo")

CONCLUSIONES PRINCIPALES:
1. DIVERSIDAD EMOCIONAL:
   Las novelas clásicas presentan una rica variedad de emociones, reflejando
   la complejidad de la experiencia humana en la literatura.

2. PATRONES LITERARIOS:
   - Las novelas románticas tienden a mostrar más emociones de "alegría" y "confianza"
   - Las novelas trágicas muestran predominancia de "tristeza" y "miedo"
   - Las novelas de aventura presentan más "anticipación" y "sorpresa"

3. BALANCE EMOCIONAL:
   La mayoría de las obras clásicas mantienen un equilibrio entre emociones positivas
   y negativas, lo cual contribuye a narrativas más complejas y realistas.

4. LIMITACIONES DEL ANÁLISIS:
   - El léxico emocional puede no capturar matices literarios complejos
   - El contexto narrativo es importante para interpretar emociones
   - La traducción y el lenguaje histórico pueden afectar la detección

5. APLICACIONES FUTURAS:
   - Análisis comparativo entre géneros literarios
   - Estudio de la evolución emocional a lo largo de una obra
   - Análisis de arcos narrativos mediante perfiles emocionales

In [ ]:
# ### Mejoras posibles:
# - Usar el léxico NRC completo (14,182 palabras)
# - Implementar análisis de contexto y negación
# - Añadir análisis de intensidad emocional
# - Considerar expresiones multi-palabra
# - Implementar análisis temporal (evolución emocional en la narrativa)
# - Añadir comparación estadística entre géneros literarios